In [61]:
import requests
import re 
import os 
import time
import traceback 
from bs4 import BeautifulSoup 
from lxml import etree
from fake_useragent import UserAgent
import pandas as pd

In [12]:
def getHtmlContent(url):
 UA = UserAgent(verify_ssl = False)   
 headers = {
    "User-Agent": UA.random
    }

 try:
    s = requests.Session()
    r = s.get(url, headers = headers, cookies = {'from-my':'browser'})
    r.raise_for_status()
    if(r.encoding != "UTF-8"):
        r.encoding = r.apparent_encoding
    return r.text
 except:
    return ""

In [13]:
URL = 'https://www.dianping.com/search/keyword/22/0_%E7%BB%98%E6%9C%AC%E9%A6%86' #22 Jinan
html = getHtmlContent(url = URL)
soup = BeautifulSoup(html, "html.parser")
soup

<html>
<head>
<link href="//www.dpfile.com/app/pc-common/dp_favicon.a4af753914321c8e82e402e2b4be01d7.ico" rel="icon" type="image/x-icon"/>
<link href="//www.dpfile.com/app/pc-common/dp_favicon.a4af753914321c8e82e402e2b4be01d7.ico" rel="shortcut icon" type="image/x-icon"/>
<title>济南绘本馆相关搜索结果推荐-大众点评网</title>
<meta content="绘本馆" name="Keywords"/>
<meta content="为您提供与绘本馆相关的商户,并且有店铺实拍图片、地址、电话、价格、时间营业时间等信息，还有其他网友的真实体验评价，帮您找到济南最符合绘本馆的内容。" name="Description"/>
<meta content="format=html5; url=https://m.dianping.com/search/keyword/22/0_%E7%BB%98%E6%9C%AC%E9%A6%86" http-equiv="mobile-agent"/>
<link href="https://m.dianping.com/search/keyword/22/0_%E7%BB%98%E6%9C%AC%E9%A6%86" media="only screen and (max-width: 640px)" rel="alternate"/>
<meta charset="utf-8">
<script type="text/javascript">
        window._DP_HeaderData = {
            'cityId': '22',                   
            'cityChName': '济南',                 
            'cityEnName': 'jinan',               
            'pageType': 'searc

In [50]:
tree = etree.HTML(str(soup))
URLlist = []
for i in range(1, 16):
    link = tree.xpath('//*[@id="shop-all-list"]/ul/li['+ str(i) + ']/div[2]/div[1]/a/@href')
    URLlist.append(link[0])



In [78]:
print(URLlist)

['https://www.dianping.com/shop/H9LR34sHvCC9Xn0h', 'https://www.dianping.com/shop/l6T62dG5LsOFHbjZ', 'https://www.dianping.com/shop/l7vgwCwPFrElKNh5', 'https://www.dianping.com/shop/l409Jgo7g3DbnkAo', 'https://www.dianping.com/shop/k7KfUOpAfQpQDccR', 'https://www.dianping.com/shop/H6LmwseysJyQNvLI', 'https://www.dianping.com/shop/H8ju8cLlBw7jVp6F', 'https://www.dianping.com/shop/k29XdDoXmN577JBL', 'https://www.dianping.com/shop/k6AArWYyt0kA6Ilm', 'https://www.dianping.com/shop/G6x135yWr4qCNDBH', 'https://www.dianping.com/shop/la9cFu0Y0R6LlD6e', 'https://www.dianping.com/shop/Ez94wklM57pKz7K5', 'https://www.dianping.com/shop/G1V9IJZvMhX5ixsW', 'https://www.dianping.com/shop/G7UgNQB798ug4J6v', 'https://www.dianping.com/shop/l4Ml7wiItVdLO6lw']


//*[@id="shop-all-list"]/ul/li[1]/div[2]/div[1]/a/h4

In [79]:
Liblist = []
for i in range(1,16):
    Libname = tree.xpath('//*[@id="shop-all-list"]/ul/li[' + str(i)+ ']/div[2]/div[1]/a/h4/text()')
    Liblist.append(Libname[0])

In [80]:
print(Liblist)

['100层亲子图书馆', '星星和蒲公英儿童图书馆', '老约翰绘本馆', '倍爱读绘本馆(融创茂店)', '倍爱读绘本馆(领秀城贵和店)', '老约翰绘本馆(济南二站)', '绘本(振华商厦店)', '全智启蒙中心(和谐校区)', '绘本(银座购物广场店)', '读小书儿童绘本馆', '氣味圖書館(万象城店)', '爱阅大眼睛童书馆(NO.SDJN0001店)', '童学馆(少儿国学教育)', '倍爱读绘本馆(全福店)', '小书虫绘本馆(官扎营后街店)']


In [90]:
JN_lib =pd.DataFrame({'Shop Name':Liblist,'URL': URLlist}) 
JN_lib

,Shop Name,URL
0,100层亲子图书馆,https://www.dianping.com/shop/H9LR34sHvCC9Xn0h
1,星星和蒲公英儿童图书馆,https://www.dianping.com/shop/l6T62dG5LsOFHbjZ
2,老约翰绘本馆,https://www.dianping.com/shop/l7vgwCwPFrElKNh5
3,倍爱读绘本馆(融创茂店),https://www.dianping.com/shop/l409Jgo7g3DbnkAo
4,倍爱读绘本馆(领秀城贵和店),https://www.dianping.com/shop/k7KfUOpAfQpQDccR
5,老约翰绘本馆(济南二站),https://www.dianping.com/shop/H6LmwseysJyQNvLI
6,绘本(振华商厦店),https://www.dianping.com/shop/H8ju8cLlBw7jVp6F
7,全智启蒙中心(和谐校区),https://www.dianping.com/shop/k29XdDoXmN577JBL
8,绘本(银座购物广场店),https://www.dianping.com/shop/k6AArWYyt0kA6Ilm
9,读小书儿童绘本馆,https://www.dianping.com/shop/G6x135yWr4qCNDBH


In [110]:
subpage_soup = []
for i in range(0,len(JN_lib)):
    subpage_soup.append(BeautifulSoup(getHtmlContent(url = JN_lib['URL'][i]), "html.parser"))
    time.sleep(3)

In [113]:
subpage_soup[0]

<!DOCTYPE html>

<html>
<head>
<link href="http://www.dianping.com/shop/H9LR34sHvCC9Xn0h" rel="canonical"/>
<script type="text/javascript">(function(){var g=["iPhone","Android","Windows Phone"],b="http://m.dianping.com",e=window.location.pathname+window.location.search,f=false;for(var d=0,a;a=g[d];d++){if(navigator.userAgent.indexOf(a)!=-1){f=true;break}}if(document.referrer.indexOf("baidu.com")>0){if(window.location.search==""){e=e+"?msource=baidusearch"}else{e=e+"&msource=baidusearch"}}else if(document.referrer.indexOf("sogou.com")>0){if(window.location.search==""){e=e+"?msource=sogousearch"}else{e=e+"&msource=sogousearch"}}function h(){location.href=b+e}function c(){if(document.referrer.indexOf(b)==0){document.cookie="vmod=pc;expires=0;"}return document.cookie.match(/vmod=pc/)}if(f){if(!c()){h()}}})();</script>
<title>100层亲子图书馆课程_价格_简介_怎么样-济南学习培训-大众点评网</title>
<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
<meta content="pc" name="applicable-device"/>
<meta content="100层亲子图书

In [114]:

Tree = []
for i in range(0, len(subpage_soup)):
    Tree.append(etree.HTML(str(subpage_soup[i])))
    

In [127]:
AddressList = []
HourList = []
PhoneNumList = []

for i in range(0, 2):
    AddressList.append(Tree[i].xpath('/html/body/div[3]/div/div[1]/div[1]/div[2]/div[2]/div[2]/text()[normalize-space()]')[0].strip())
    PhoneNumList.append(Tree[i].xpath('/html/body/div[3]/div/div[1]/div[1]/div[2]/div[2]/div[4]/div/span/text()[normalize-space()]')[0].strip())
    HourList.append(Tree[i].xpath('//*[@id="info"]/ul/li[3]/text()[normalize-space()]')[0].strip())



In [128]:
AddressList

['舜宁路29号', '洪山路金邸山庄2号楼底商']